### 실시간 영상에 분류 결과 나타내기

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

In [2]:
CHECK_POINT_DIR = './model.ckpt'

In [3]:
checkpoint = tf.train.get_checkpoint_state(CHECK_POINT_DIR)
if checkpoint :
    print(checkpoint)
    print(checkpoint.model_checkpoint_path)

model_checkpoint_path: "./model.ckpt/model"
all_model_checkpoint_paths: "./model.ckpt/model"

./model.ckpt/model


In [4]:
# initialize
config = ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [5]:
saver = tf.train.import_meta_graph(checkpoint.model_checkpoint_path + '.meta')

In [6]:
sess.run(tf.global_variables_initializer())
if checkpoint and checkpoint.model_checkpoint_path :
    try : 
        # saver = tf.train.import_meta_graph(checkpoint.model_checkpoint_path + '.meta')
        saver.restore(sess, checkpoint.model_checkpoint_path)
        print('Successfully loaded : ', checkpoint.model_checkpoint_path)
    except :
        print("Error on loading old network weights")
else :
    print("Could not find old network weights")

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./model.ckpt/model
Successfully loaded :  ./model.ckpt/model


In [7]:
LABELS = []
for folder in glob.glob('./data/*') :
    label = folder.split('/')[-1]
    # label = label.split('\\')[-1]
    # print(label)
    LABELS.append(label)
print(LABELS)

['70', 'no_parking', 'no_parking_stop', '50', '60', '30']


In [8]:
# 클래스 개수
N_CLASSES = len(LABELS)
# print(N_CLASSES)
# input image size
RESIZED_IMAGE = (28, 28)

In [9]:
IDX_LABEL = {}
for i, label in zip(range(0, N_CLASSES), LABELS) :
    IDX_LABEL[i] = label
print(IDX_LABEL)

{0: '70', 1: 'no_parking', 2: 'no_parking_stop', 3: '50', 4: '60', 5: '30'}


In [10]:
# 모델 생성 함수
def build_model(X) :
    
    # Layer1 / ImgIn shape = (?, 28, 28, 1)
    # kernel_size = [3, 3], filters = 32
    W1 = tf.Variable(tf.random_normal([3, 3, 1, 32], stddev = 0.01))
    L1 = tf.nn.conv2d(X, W1, strides = [1, 1, 1, 1], padding = 'SAME')
    L1 = tf.nn.relu(L1)
    L1 = tf.nn.max_pool(L1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    L1 = tf.nn.dropout(L1, keep_prob = keep_prob)
    
    # Layer2 / ImgIn shape = (?, 14, 14, 32)
    # kernel_size = [3, 3], filters = 64 
    W2 = tf.Variable(tf.random_normal([3, 3, 32, 64], stddev = 0.01))
    L2 = tf.nn.conv2d(L1, W2, strides = [1, 1, 1, 1], padding = 'SAME')
    L2 = tf.nn.relu(L2)
    L2 = tf.nn.max_pool(L2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    L2 = tf.nn.dropout(L2, keep_prob = keep_prob)
    
    # Layer3 / ImgIn shape = (?, 7, 7, 64)
    W3 = tf.Variable(tf.random_normal([3, 3, 64, 128], stddev = 0.01))
    L3 = tf.nn.conv2d(L2, W3, strides = [1, 1, 1, 1], padding = 'SAME')
    L3 = tf.nn.relu(L3)
    L3 = tf.nn.max_pool(L3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1], padding = 'SAME')
    L3 = tf.nn.dropout(L3, keep_prob = keep_prob)
    L3_flat = tf.reshape(L3, [-1, 128 * 4 * 4])
    
    # Layer4 / FC 4x4x128 inputs -> 625 outputs
    W4 = tf.get_variable("W4", shape = [128 * 4 * 4, 625], initializer=tf.contrib.layers.xavier_initializer())
    b4 = tf.Variable(tf.random_normal([625]))
    L4 = tf.nn.relu(tf.matmul(L3_flat, W4) + b4)
    L4 = tf.nn.dropout(L4, keep_prob = keep_prob)
    
    # Layer5 / Final FC 625 inputs -> (N_CLASSES) outputs
    W5 = tf.get_variable("W5", shape = [625, N_CLASSES], initializer=tf.contrib.layers.xavier_initializer())
    b5 = tf.Variable(tf.random_normal([N_CLASSES]))
    logits = tf.matmul(L4, W5) + b5
    y_pred = tf.nn.softmax(logits)
    
    return y_pred, logits

In [11]:
# dropout (keep_prob) rate  0.7~0.5 on training, but should be 1 for testing
keep_prob = tf.placeholder(tf.float32)

# input placeholders
X = tf.placeholder(tf.float32, [None, RESIZED_IMAGE[0], RESIZED_IMAGE[1], 1])
y = tf.placeholder(tf.float32, [None, N_CLASSES])

In [12]:
y_pred, logits = build_model(X)

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [13]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
from sklearn.metrics import classification_report, confusion_matrix
import random
import matplotlib.pyplot as plt
import glob
import numpy as np
import cv2
import os
from collections import namedtuple
%matplotlib inline

In [14]:
LABEL_IDX = {}
for label, i in zip(LABELS, range(0, N_CLASSES)) :
    LABEL_IDX[label] = i
print(LABEL_IDX)

{'70': 0, 'no_parking': 1, 'no_parking_stop': 2, '50': 3, '60': 4, '30': 5}


In [15]:
Dataset = namedtuple('Dataset', ['X', 'y'])
print(Dataset)

<class '__main__.Dataset'>


In [16]:
# 텐서플로우에 맞는 이미지 텐서 포맷 : [?, ?, ?, ?]
# 차원 확장
def to_tf_format(imgs) :
    return np.stack([img[:, :, np.newaxis] for img in imgs], axis = 0).astype(np.float32)

In [17]:
def read_dataset(root_path, labels_arr, resize_to) :
    images = []
    labels = []
    
    for label_name in labels_arr :
        # 'data/no_parking/'
        full_path = root_path + '/' + label_name + '/'
        
        for img in glob.glob(full_path + '*.jpg') :
            image = cv2.imread(img).astype(np.float32)

            # 회색조로 변환
            image = cv2.cvtColor(image, cv2.IMREAD_GRAYSCALE)[:, :, 0]
            
            # 크기 조정
            if resize_to : 
                image = cv2.resize(image, resize_to)
            
            # 라벨
            # 원-핫 인코딩
            label = np.zeros((N_CLASSES, ), dtype = np.float32)
            label[LABEL_IDX[label_name]] = 1.0
            
            images.append(image.astype(np.float32))
            labels.append(label)
            
    return Dataset(X = to_tf_format(images).astype(np.float32), y = np.matrix(labels).astype(np.float32))

In [18]:
dataset = read_dataset('data', LABELS, RESIZED_IMAGE)

In [19]:
from sklearn.model_selection import train_test_split

idx_train, idx_test = train_test_split(range(dataset.X.shape[0]), test_size = 0.25, random_state = 15 )

In [20]:
X_train = dataset.X[idx_train, :, :, :]
X_test = dataset.X[idx_test, :, :, :]
y_train = dataset.y[idx_train, :]
y_test = dataset.y[idx_test, :]
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(2334, 28, 28, 1)
(779, 28, 28, 1)
(2334, 6)
(779, 6)


In [21]:
print(classification_report(sess.run(tf.argmax(y_test, 1)), 
                            sess.run(tf.argmax(y_pred, 1), feed_dict = {X : X_test, keep_prob : 1})))

FailedPreconditionError: 2 root error(s) found.
  (0) Failed precondition: Attempting to use uninitialized value Variable_5
	 [[node Variable_5/read (defined at <ipython-input-10-9c281e2634a0>:6) ]]
	 [[ArgMax_1/_53]]
  (1) Failed precondition: Attempting to use uninitialized value Variable_5
	 [[node Variable_5/read (defined at <ipython-input-10-9c281e2634a0>:6) ]]
0 successful operations.
0 derived errors ignored.

Original stack trace for 'Variable_5/read':
  File "/home/team2/anaconda3/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/team2/anaconda3/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/team2/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/team2/anaconda3/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/team2/anaconda3/lib/python3.7/si

In [ ]:
# 영상 경로 설정
video_path = './video_data/video1.mp4'
cap = cv2.VideoCapture(video_path)

video_file = video_path.split('/')[-1]
video_name = video_file.split('.')[0]

# 커널 생성
kernel1 = np.ones((2, 2), np.uint8)
kernel2 = np.ones((4, 4), np.uint8)

# 프레임 이동
# frame_move = int(cap.get(cv2.CAP_PROP_FRAME_COUNT) // 2)
# cap.set(cv2.CAP_PROP_POS_FRAMES, frame_move)

while cap.isOpened() :
    ret, img = cap.read()
    
    if ret is False :
        break
        
    # 이미지 크기 변경
    img = cv2.resize(img, (img.shape[1]//2, img.shape[0]//2), interpolation = cv2.INTER_AREA)
    
    ######### 1. 표지판 검출 #########
    
    ## 1-1. 표지판 ROI 생성
    img_sign = np.zeros(img.shape, img.dtype)
    img_sign[0 : img.shape[0]*11//20, : ] = img[0 : img.shape[0]*11//20 ,  : ]
       
    ## 1-2. 붉은색 검출
    # 1) 색 공간 변환(BGR2YUV) -> 붉은색 계열 검출
    yuv_sign = cv2.cvtColor(img_sign, cv2.COLOR_BGR2YUV)
    yuv_sign[yuv_sign[:, :, 2]<135] = 0 # V 채널
    yuv_sign[yuv_sign[:, :, 1]<110] = 0 # U 채널
    yuv_sign[yuv_sign[:, :, 0]>225] = 0 # 밝기
    # V 채널 이진화 
    th, sign_v_bin = cv2.threshold(yuv_sign[ : , : , 2], 140, 255, cv2.THRESH_BINARY)
    erosion_sign_v_bin = cv2.erode(sign_v_bin, kernel1, iterations = 1) # 침식
    dilation_sign_v_bin = cv2.dilate(erosion_sign_v_bin, kernel2, iterations = 1) # 팽창
    
    # 2) 캐니 엣지 -> 조명 영향 제거
    sign_edge = cv2.Canny(img_sign, 100, 200)
    dilation_sign_edge = cv2.dilate(sign_edge, kernel1, iterations = 1) # 팽창
    erosion_sign_edge = cv2.erode(dilation_sign_edge, kernel1, iterations = 1) # 침식
    
    sign_red = cv2.bitwise_and(dilation_sign_v_bin, dilation_sign_v_bin, mask = erosion_sign_edge)
    
    ## 1-3. 원 검출
    # 1) 컨투어
    # RETR_EXTERNAL / RETR_TREE / RETR_LIST / RETR_CCOMP
    _, contours, hierarchy = cv2.findContours(sign_red, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)
    
    mask_contours = np.zeros(sign_red.shape, sign_red.dtype)
    
    for contour in contours :
        epsilon = 0.005 * cv2.arcLength(contour, True)
        # 근사 컨투어
        approx = cv2.approxPolyDP(contour, epsilon, closed = True)
        cv2.drawContours(mask_contours, [approx], -1, 255, 2)
    
    # 2) 객체 검출
    _, _, stats, centroids = cv2.connectedComponentsWithStats(mask_contours)    
    # cv2.imshow('mask_contours', mask_contours)
    
    for idx, centroid in enumerate(centroids) :
        if stats[idx][0] == 0 and stats[idx][1] == 0 :
            continue
        if np.any(np.isnan(centroid)) :
            continue
            
        x, y, w, h, area = stats[idx]
        centerX, centerY = int(centroid[0]), int(centroid[1])
        
        if area<1700 and area>100 and abs(w-h)< 5 :
            # try : 
            detected_img = img[y-5 : y+h+5,  x-5 : x+w+5]
            gray_detected_img = cv2.cvtColor(detected_img, cv2.COLOR_BGR2GRAY)

            # 3) 허프 원 검출
            circles = cv2.HoughCircles(gray_detected_img, cv2.HOUGH_GRADIENT, 1, 100, param1=200, param2=40, minRadius=3, maxRadius=38)
            if circles is not None :
                circles = np.uint16(np.around(circles))

                for i in circles[0, :] :                      
                    frame = img[y-5 : y+h+5,  x-5 : x+w+5]
##############################################################################################                        
                    # bgr -> rgb 로 배열순서 변경
                    # frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    # 리사이즈 : 28 * 28
                    frame = cv2.cvtColor(frame, cv2.IMREAD_GRAYSCALE)[:, :, 0]
                    frame = cv2.resize(frame, (28, 28))
                    frame = frame[:, :, np.newaxis]
                    frame = np.expand_dims(frame, axis=0)
                    # print(frame.shape)
                    # 예측
                    # pred = model.predict(np.expand_dims(frame, axis=0))[0]
                    pred = sess.run(tf.argmax(y_pred, axis = 1), feed_dict = {X : frame, keep_prob : 1})
                    print(pred)
                    # 레이블 확인
                    # label = lb.classes_[np.argmax(pred)]
                    label = IDX_LABEL[pred[0]]
##############################################################################################                        
                    text = "{}".format(label)
                    cv2.putText( img, text, ( x, y + 100 ), cv2.FONT_HERSHEY_SIMPLEX, 1.0, ( 0, 255, 0 ), 3)
                    cv2.rectangle(img, (x-5, y-5), (x+w+5, y+h+5), (0, 0, 255), 2)
            # except : 
                # continue
    
    ######### 2. 차선 검출 #########
    
    ## 2-1. 차선 ROI 생성
    img_road = np.zeros(img.shape, img.dtype)
    img_road[0 : img.shape[0]*11//20, : ] = img[0 : img.shape[0]*11//20 ,  : ]    
    
    ###############################
    
    cv2.imshow(video_name, img)
    
    if cv2.waitKey(1) == 27 :
        cap.release()
        cv2.destroyAllWindows()
    
cap.release()
cv2.destroyAllWindows()